In [5]:
import pandas as pd
import os
import datetime
import pickle
import re
import time

In [6]:
# Reg relevant articles
df=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/RegSentsExpand_NounChunks_Update.pkl')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9278 entries, 0 to 9277
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           9278 non-null   object        
 1   Title                        9278 non-null   object        
 2   Type                         9278 non-null   object        
 3   StartDate                    9278 non-null   datetime64[ns]
 4   EndDate                      9278 non-null   object        
 5   Text                         9278 non-null   object        
 6   TextWordCount                9278 non-null   object        
 7   PubTitle                     9278 non-null   object        
 8   SourceType                   9278 non-null   object        
 9   Year                         9278 non-null   int64         
 10  Month                        9278 non-null   int64         
 11  Newspaper                    9278 non-null 

In [7]:
# Use filtered noun chunk matches to define reg relevance
df.loc[df['NounChunkMatchFiltered']!=0,'RegRelevance']=1
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9278 entries, 0 to 9277
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           9278 non-null   object        
 1   Title                        9278 non-null   object        
 2   Type                         9278 non-null   object        
 3   StartDate                    9278 non-null   datetime64[ns]
 4   EndDate                      9278 non-null   object        
 5   Text                         9278 non-null   object        
 6   TextWordCount                9278 non-null   object        
 7   PubTitle                     9278 non-null   object        
 8   SourceType                   9278 non-null   object        
 9   Year                         9278 non-null   int64         
 10  Month                        9278 non-null   int64         
 11  Newspaper                    9278 non-null 

In [8]:
# LM uncertainty scores
LMuncertainty=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Update Data/LMuncertainty_Update.csv')
print(LMuncertainty.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                5832 non-null   int64 
 1   UncertaintyCount  5832 non-null   int64 
 2   UncertaintyWords  5832 non-null   object
dtypes: int64(2), object(1)
memory usage: 136.8+ KB
None


In [9]:
LMuncertainty['ID']=LMuncertainty['ID'].astype('str')
print(LMuncertainty.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                5832 non-null   object
 1   UncertaintyCount  5832 non-null   int64 
 2   UncertaintyWords  5832 non-null   object
dtypes: int64(1), object(2)
memory usage: 136.8+ KB
None


In [10]:
# Merge
df2=df.merge(LMuncertainty,on='ID',how='right')
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832 entries, 0 to 5831
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           5832 non-null   object        
 1   Title                        5832 non-null   object        
 2   Type                         5832 non-null   object        
 3   StartDate                    5832 non-null   datetime64[ns]
 4   EndDate                      5832 non-null   object        
 5   Text                         5832 non-null   object        
 6   TextWordCount                5832 non-null   object        
 7   PubTitle                     5832 non-null   object        
 8   SourceType                   5832 non-null   object        
 9   Year                         5832 non-null   int64         
 10  Month                        5832 non-null   int64         
 11  Newspaper                    5832 non-null 

In [11]:
# GI sentiments
GIsentiments=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Update Data/GIsentiments_Update.csv')
GIsentiments['ID']=GIsentiments['ID'].astype('str')
print(GIsentiments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              5832 non-null   object
 1   TotalWordCount  5832 non-null   int64 
 2   GIposCount      5832 non-null   int64 
 3   GInegCount      5832 non-null   int64 
 4   GIposWords      5832 non-null   object
 5   GInegWords      5832 non-null   object
dtypes: int64(3), object(3)
memory usage: 273.5+ KB
None


In [12]:
# LSD sentiments
LSDsentiments=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Update Data/LSDsentiments_Update.csv')
LSDsentiments['ID']=LSDsentiments['ID'].astype('str')
print(LSDsentiments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           5832 non-null   object
 1   LSDposCount  5832 non-null   int64 
 2   LSDnegCount  5832 non-null   int64 
 3   LSDposWords  5832 non-null   object
 4   LSDnegWords  5832 non-null   object
dtypes: int64(2), object(3)
memory usage: 227.9+ KB
None


In [13]:
# LM sentiments
LMsentiments=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Update Data/LMsentiments_Update.csv')
LMsentiments['ID']=LMsentiments['ID'].astype('str')
print(LMsentiments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          5832 non-null   object
 1   LMposCount  5832 non-null   int64 
 2   LMnegCount  5832 non-null   int64 
 3   LMposWords  5832 non-null   object
 4   LMnegWords  5832 non-null   object
dtypes: int64(2), object(3)
memory usage: 227.9+ KB
None


In [14]:
# Merge
df3=df2.merge(GIsentiments,on='ID',how='left').\
    merge(LMsentiments,on='ID',how='left').\
    merge(LSDsentiments,on='ID',how='left')
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832 entries, 0 to 5831
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           5832 non-null   object        
 1   Title                        5832 non-null   object        
 2   Type                         5832 non-null   object        
 3   StartDate                    5832 non-null   datetime64[ns]
 4   EndDate                      5832 non-null   object        
 5   Text                         5832 non-null   object        
 6   TextWordCount                5832 non-null   object        
 7   PubTitle                     5832 non-null   object        
 8   SourceType                   5832 non-null   object        
 9   Year                         5832 non-null   int64         
 10  Month                        5832 non-null   int64         
 11  Newspaper                    5832 non-null 

In [15]:
# Reg relevant articles only
df_reg=df3[df3['RegRelevance']==1]
print(df_reg.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832 entries, 0 to 5831
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           5832 non-null   object        
 1   Title                        5832 non-null   object        
 2   Type                         5832 non-null   object        
 3   StartDate                    5832 non-null   datetime64[ns]
 4   EndDate                      5832 non-null   object        
 5   Text                         5832 non-null   object        
 6   TextWordCount                5832 non-null   object        
 7   PubTitle                     5832 non-null   object        
 8   SourceType                   5832 non-null   object        
 9   Year                         5832 non-null   int64         
 10  Month                        5832 non-null   int64         
 11  Newspaper                    5832 non-null 

In [16]:
# Calculate scores
df_reg['UncertaintyScore']=df_reg['UncertaintyCount']/df_reg['TotalWordCount']*100
for dic in ['GI','LSD','LM']:
    df_reg[dic+'score']=(df_reg[dic+'posCount']-df_reg[dic+'negCount'])/df_reg['TotalWordCount']*100
print(df_reg.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832 entries, 0 to 5831
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           5832 non-null   object        
 1   Title                        5832 non-null   object        
 2   Type                         5832 non-null   object        
 3   StartDate                    5832 non-null   datetime64[ns]
 4   EndDate                      5832 non-null   object        
 5   Text                         5832 non-null   object        
 6   TextWordCount                5832 non-null   object        
 7   PubTitle                     5832 non-null   object        
 8   SourceType                   5832 non-null   object        
 9   Year                         5832 non-null   int64         
 10  Month                        5832 non-null   int64         
 11  Newspaper                    5832 non-null 

In [13]:
# Export all data
df_reg.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/RegRelevant_ArticleSentimentWordsScores_Update.pkl')

In [18]:
# Export sentiment scores only
sentimentScores=df_reg[['ID','StartDate','Newspaper','UncertaintyScore','GIscore','LMscore','LSDscore']]
print(sentimentScores.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832 entries, 0 to 5831
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                5832 non-null   object        
 1   StartDate         5832 non-null   datetime64[ns]
 2   Newspaper         5832 non-null   object        
 3   UncertaintyScore  5832 non-null   float64       
 4   GIscore           5832 non-null   float64       
 5   LMscore           5832 non-null   float64       
 6   LSDscore          5832 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 364.5+ KB
None


In [15]:
sentimentScores.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Update Data/RegRelevant_ArticleSentimentScores_Update.csv',index=False)